# Hyper Parameter Tunning (Sklearn)

머신러닝 알고리즘을 통해 학습을 진행할 때, 알고리즘의 다양한 파라미터를 통해 데이터에 대한 최적의 모델을 찾는 과정이 필요합니다. 메뉴얼한 방식으로 대략적인 알고리즘의 선택을 할 수는 있지만, 최적의 모델을 선택하기 위해서는 다양한 파라미터 조합을 통한 많은 학습이 필요하게 됩니다. 데이터 수, 피처 수, 파라미터 조합에 따라 최적화 수행 시간은 기하 급수적으로 늘어나게 됩니다.

## Scikit-Learn

### Grid Search & Randomized Search

파라미터에 의해서 생성할 수 있는 모든 파라미터의 조합을 생성하여 최적화 파라미터를 찾습니다. 전수를 테스트하는 Grid Search 방식은 Search Space 가 크기 때문에 조금만 파라미터가 커치면 학습해야하는 조합이 늘게 됩니다. 그래서 이보다는 모든 조합을 시도하는 대신 모든 반복에서 각 파라미터에 대해 랜덤 값을 선택하여 학습 조합 수를 줄이는 Randomized Search 방식이 자주 사용됩니다.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import numpy as np
from sklearn import datasets, model_selection, svm, metrics
import os, joblib

# 데이터
np.random.seed(0)
n_samples = 1000
np_data_xs, np_data_ys = datasets.make_classification(
    n_samples=n_samples, # 데이터 수
    n_features=10, # X feature 수
    n_informative=3,
    n_classes=3, # Y class 수
    random_state=0) # 난수 발생용 Seed 값
print("data shape: np_data_xs={}, np_data_ys={}".format(np_data_xs.shape, np_data_ys.shape))
np_train_xs, np_test_xs, np_train_ys, np_test_ys = model_selection.train_test_split(
    np_data_xs, np_data_ys, 
    test_size=0.3, shuffle=True, random_state=2)

# Set the parameters by cross-validation
tuned_parameters = [
    {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]},
    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for {}".format(score))
    print()

    #clf = model_selection.GridSearchCV(svm.SVC(), tuned_parameters, scoring='%s_macro' % score, n_jobs=-1)
    clf = model_selection.RandomizedSearchCV(svm.SVC(), tuned_parameters, scoring='%s_macro' % score, n_jobs=-1)
    
    clf.fit(np_train_xs, np_train_ys)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Best estimator set found on development set:")
    print()
    print(clf.best_estimator_)
    print()    
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = np_test_ys, clf.predict(np_test_xs)
    print(metrics.classification_report(y_true, y_pred))
    print()

data shape: np_data_xs=(1000, 10), np_data_ys=(1000,)
# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}

Best estimator set found on development set:

SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Grid scores on development set:

0.895 (+/-0.043) for {'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}
0.880 (+/-0.042) for {'kernel': 'linear', 'C': 1000}
0.885 (+/-0.055) for {'kernel': 'rbf', 'gamma': 0.001, 'C': 100}
0.880 (+/-0.042) for {'kernel': 'linear', 'C': 100}
0.872 (+/-0.076) for {'kernel': 'rbf', 'gamma': 0.001, 'C': 10}
0.876 (+/-0.049) for {'kernel': 'linear', 'C': 1}
0.443 (+/-0.043) for {'kernel': 'rbf', 'gamma': 0.0001, 'C': 1}
0.882 (+/-0.056) for {'kernel': 'rbf', 'gamma': 0.0001, 'C': 1000}
0.